In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from ultralytics import YOLO
from tqdm import tqdm
import os
%matplotlib inline  

In [2]:
# Load the models
control_model = YOLO('last_control.pt')

In [3]:
metrics = control_model.val(data="C:\Repositories\OpenUniversiteit\multispectral-ped-detection\config_control.yaml", split="test")

Ultralytics YOLOv8.0.226 🚀 Python-3.10.11 torch-2.1.1+cpu CPU (Intel Core(TM) i7-8550U 1.80GHz)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning B:\multispectral-ped-detection\data\DOTA_CONTROL\labels\test... 182 images, 0 backgrounds, 0 corrupt: 100%|██████████| 182/182 [00:04<00:00, 44.36it/s]


val: New cache created: B:\multispectral-ped-detection\data\DOTA_CONTROL\labels\test.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:33<00:00,  2.82s/it]


                   all        182        182      0.799      0.802      0.858      0.545
                person        182        182      0.799      0.802      0.858      0.545
Speed: 2.4ms preprocess, 162.2ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\val


In [4]:
pr_curve = metrics.curves_results[0]
recall = pr_curve[0]
precision = pr_curve[1]
person_precision = precision[0]
people_precision = precision[1]
cyclist_precision = precision[2]

pr_curve_df = pd.DataFrame()
pr_curve_df["recall"] = recall
pr_curve_df["person_precision"] = person_precision
pr_curve_df["people_precision"] = people_precision
pr_curve_df["cyclist_precision"] = cyclist_precision
pr_curve_df.to_csv(os.path.join(metrics.save_dir, "pr_curve_results.csv"), index=False)

In [5]:
f1_curve = metrics.curves_results[1]
confidence = f1_curve[0]
f1_score = f1_curve[1]
person_f1 = f1_score[0]
people_f1 = f1_score[1]
cyclist_f1 = f1_score[2]

f1_curve_df = pd.DataFrame()
f1_curve_df["confidence"] = confidence
f1_curve_df["person_f1"] = person_f1
f1_curve_df["people_f1"] = people_f1
f1_curve_df["cyclist_f1"] = cyclist_f1
f1_curve_df.to_csv(os.path.join(metrics.save_dir, "f1_curve_results.csv"), index=False)

In [6]:
precision_curve = metrics.curves_results[2]
confidence = precision_curve[0]
precision = precision_curve[1]
person_precision = precision[0]
people_precision = precision[1]
cyclist_precision = precision[2]

precision_curve_df = pd.DataFrame()
precision_curve_df["confidence"] = confidence
precision_curve_df["person_precision"] = person_precision
precision_curve_df["people_precision"] = people_precision
precision_curve_df["cyclist_precision"] = cyclist_precision
precision_curve_df.to_csv(os.path.join(metrics.save_dir, "precision_curve_results.csv"), index=False)

In [7]:
recall_curve = metrics.curves_results[3]
confidence = recall_curve[0]
recall = recall_curve[1]
person_recall = recall[0]
people_recall = recall[1]
cyclist_recall = recall[2]

recall_curve_df = pd.DataFrame()
recall_curve_df["confidence"] = confidence
recall_curve_df["person_recall"] = person_recall
recall_curve_df["people_recall"] = people_recall
recall_curve_df["cyclist_recall"] = cyclist_recall
recall_curve_df.to_csv(os.path.join(metrics.save_dir, "recall_curve_results.csv"), index=False)

In [8]:
metrics_df = pd.DataFrame({key: [value] for key,value in metrics.results_dict.items()})

In [9]:
metrics_df["metrics/mAP50_person"] = [metrics.box.ap50[0]]
metrics_df["metrics/mAP50_people"] = [metrics.box.ap50[1]]
metrics_df["metrics/mAP50_cyclist"] = [metrics.box.ap50[2]]

metrics_df["metrics/mAP50-95_person"] = [metrics.box.ap[0]]
metrics_df["metrics/mAP50-95_people"] = [metrics.box.ap[1]]
metrics_df["metrics/mAP50-95_cyclist"] = [metrics.box.ap[2]]

In [10]:
metrics_df.to_csv(os.path.join(metrics.save_dir, "mean_results.csv"), index=False)